In [3]:
import os

dir = 'D:\\CVRPLIB-2025\\results\\hgs\\XLTEST-n6034-k1234.vrp.sol'
filename = os.path.basename(dir)
filename

'XLTEST-n6034-k1234.vrp.sol'

In [9]:
import re
from pathlib import Path

# --- 1. 设置目标目录 ---
# 请将 "your_target_directory" 替换为您的.log.txt文件所在的文件夹路径
# 例如: target_dir = Path("/home/user/my_logs")
# 或者在Windows上: target_dir = Path(r"C:\Users\YourUser\Documents\Logs")
target_dir = Path("D:\\CVRPLIB-2025\\results\\hgs")

# --- 2. 编译正则表达式 ---
# 匹配: | T(s) {target_float_time} |
# re.compile 预编译正则表达式，提高效率
# \s* 匹配任意数量的空白字符（空格、tab等）
# \s+ 匹配一个或多个空白字符
# ([\d.]+) 这是一个捕获组 (group 1)，匹配一个或多个数字或小数点（即浮点数）
time_pattern = re.compile(r"\|\s*T\(s\)\s+([\d.]+)\s*\|")

# 匹配: | Feas 41 {target_fitness} 126829.60 |
# ([\d.]+) 是第二个捕获组 (group 1)，匹配 {target_fitness}
# 126829\.60 我们使用 \. 来转义小数点，表示它是一个字面上的点，而不是"任意字符"
fitness_pattern = re.compile(r"\|\s*Feas\s+[\d.]+\s+([\d.]+)\s+[\d.]+\s*\|")

filename_dict = {
    '1048.log.txt': 'XLTEST-n1048-k139.vrp.csv',
    '2168.log.txt': 'XLTEST-n2168-k625.vrp.csv',
    '3101.log.txt': 'XLTEST-n3101-k685.vrp.csv',
    '4245.log.txt': 'XLTEST-n4245-k164.vrp.csv',
    '5174.log.txt': 'XLTEST-n5174-k170.vrp.csv',
    '5649.log.txt': 'XLTEST-n5649-k365.vrp.csv',
    '6034.log.txt': 'XLTEST-n6034-k1234.vrp.csv',
    '8575.log.txt': 'XLTEST-n8575-k343.vrp.csv',
    
}

# --- 3. 检查目录是否存在 ---
if not target_dir.is_dir():
    print(f"错误: 目录不存在: {target_dir}")
    print("请修改 'target_dir' 变量为正确的路径。")
else:
    print(f"开始处理目录: {target_dir.resolve()}")

    # --- 4. 遍历目录中所有 .log.txt 文件 ---
    # .glob("*.log.txt") 会找到所有以此结尾的文件
    for log_file_path in target_dir.glob("*.log.txt"):
        print(f"  正在处理文件: {log_file_path.name}")
        
        filename = os.path.basename(log_file_path)
        
        # --- 5. 定义输出的CSV文件名 ---
        # "文件名.csv" - 我们将 .log.txt 替换为 .csv
        # 例如: 'my_run.log.txt' -> 'my_run.log.csv'
        csv_file_path = filename_dict[filename]
        
        # 如果您确实需要 'my_run.log.txt.csv' 这种文件名，请使用下面这行代替:
        # csv_file_path = log_file_path.parent / (log_file_path.name + ".csv")
        
        # 用于临时存储找到的值
        current_time = None
        current_fitness = None
        
        try:
            # --- 6. 打开日志文件和CSV文件 ---
            # 使用 'with' 语句确保文件被正确关闭
            # encoding='utf-8' 是一个好的实践，以防日志中有特殊字符
            with open(log_file_path, 'r', encoding='utf-8') as log_file, \
                 open(csv_file_path, 'a+', encoding='utf-8') as csv_file:
                
                # 可选：写入CSV表头
                # csv_file.write("Time(s);Fitness\n")

                # --- 7. 逐行读取日志文件 ---
                for line in log_file:
                    
                    # 7.1 尝试匹配时间
                    time_match = time_pattern.search(line)
                    if time_match:
                        # 找到了，提取第一个捕获组 ([\d.]+)
                        current_time = time_match.group(1)
                        # 假设时间和fitness在不同行，继续读取下一行
                    
                    # 7.2 如果没匹配到时间，尝试匹配fitness
                    fitness_match = fitness_pattern.search(line)
                    if fitness_match:
                        # 找到了，提取第一个捕获组 ([\d.]+)
                        current_fitness = fitness_match.group(1)

                    # 7.3 检查是否时间和fitness都已找到
                    # 这个逻辑可以处理 T(s) 和 Feas 行的任意顺序
                    # 只要两个值都被赋过值（不是None），就写入并重置
                    if current_time is not None and current_fitness is not None:
                        # 7.4 写入CSV文件
                        print(f"{current_time};{current_fitness}")
                        csv_file.write(f"{current_time};{current_fitness}\n")
                        
                        # 7.5 重置变量，以便查找下一对值
                        current_time = None
                        current_fitness = None
            
            print(f"    -> 已生成CSV: {csv_file_path}")

        except Exception as e:
            print(f"    处理文件 {log_file_path.name} 时发生错误: {e}")

    print("所有文件处理完毕。")

开始处理目录: D:\CVRPLIB-2025\results\hgs
  正在处理文件: 1048.log.txt
14.81;126323.00
19.23;126323.00
23.42;126151.00
27.31;126086.00
31.27;126013.00
35.54;125848.00
39.56;125806.00
43.63;125723.00
47.66;125645.00
51.66;125645.00
55.47;125588.00
59.30;125518.00
63.17;125518.00
66.93;125476.00
70.89;125434.00
74.85;125433.00
78.77;125409.00
82.57;125365.00
86.27;125277.00
90.07;125277.00
93.66;125234.00
97.35;125208.00
100.78;125173.00
104.40;125162.00
107.97;125162.00
111.80;125120.00
115.49;125120.00
118.96;125118.00
122.34;125104.00
125.78;125082.00
129.18;125056.00
132.99;125056.00
136.63;125044.00
140.34;125030.00
145.29;125022.00
149.88;125003.00
154.65;124994.00
159.07;124994.00
163.66;124987.00
168.17;124962.00
172.84;124954.00
177.37;124948.00
181.61;124933.00
186.26;124923.00
190.75;124923.00
195.25;124899.00
199.59;124872.00
203.94;124869.00
208.11;124851.00
212.59;124846.00
216.75;124829.00
221.10;124821.00
225.26;124818.00
229.41;124817.00
233.85;124813.00
238.16;124813.00
242.46;1248